In [1]:
"""
This notebook will be used as a template for each dataset giving the key outputs I need for presentations
and publications
"""

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from methods_of_Dcalc import Dpointder, Dlinalg, Dlinalgw, Dpointder3D, Dlinalg3D, Dlinalgw3D

import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as opt
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
# Type in datasets you would like to combine.  Adjust totvids to reflect the total number of videos. Also type in the
# umppx and fps in the conversion dictionary.

#Use 40 frames for 200ms and 30 frames for 300ms.
#Use 10 frames for 3D datasets

logplot = 'CAM_40x_3D_1um_logplot'
Mplot = 'CAM_40x_3D_1um_Mplot'
Dplot = 'CAM_40x_3D_1um_Dplot'
Hplot = 'CAM_40x_3D_1um_Hplot'

conversion = dict() #First element is the umppx, second is fps

conversion[1] = (0.16, 0.54, 1)
conversion[2] = (0.16, 0.54, 1)
conversion[3] = (0.16, 0.54, 1)

trajectory = dict()

trajectory[1] = np.genfromtxt('../../sample_data/12-07-06-75p-glycerol-exp/CAM_40x_3D_1um_2_T.csv',
            delimiter =",")
trajectory[2] = np.genfromtxt('../../sample_data/12-07-06-75p-glycerol-exp/CAM_40x_3D_1um_2_T.csv',
            delimiter =",")
trajectory[3] = np.genfromtxt('../../sample_data/12-07-06-75p-glycerol-exp/CAM_40x_3D_1um_2_T.csv',
            delimiter =",")

totvids = 3

for num in range(1, totvids + 1):
    
    #Remove titles from columns
    #trajectory[num]=np.delete(trajectory[num], 0,0)
    #Remove the number column from dataset
    trajectory[num]=np.delete(trajectory[num],0,1)

#Get rid of trajectories that are too short, as determined by prettify
final = dict()
tots = dict()

for num in range(1, totvids + 1):
    (final[num], tots[num]) = prettify(trajectory[num], 15, 15, conversion[num][0], conversion[num][1], conversion[num][2])


#Collect trajectories of separate videos into one array
parts = dict()
lehi = dict()

for num in range(1, totvids + 1):
    parts[num] = tots[num]
    lehi[num] = final[num][1]
    counter = 1

    while counter < parts[num]:
        counter = counter + 1
        lehi[num] = np.append(lehi[num], final[num][counter], axis=0)

for num in range(2, totvids + 1):
    lehi[num][:,0] = lehi[num][:,0] + max(lehi[num-1][:, 0])        
        
nephi = lehi[1]
counter = 1

while counter < totvids:
    counter = counter + 1
    nephi = np.append(nephi, lehi[counter], axis=0)

In [3]:
D = dict()
SD = dict()
total = dict()
t = dict()

D['pd3D'], SD['pd3D'], total['pd3D'], t['pd3D'] = Dpointder3D(lehi[1], 0, 15, 16, 1, '3D')
D['pd2Dxy'], SD['pd2Dxy'], total['pd2Dxy'], t['pd2Dxy'] = Dpointder3D(lehi[1], 0, 15, 16, 1, '2Dxy')
D['pd2Dxz'], SD['pd2Dxz'], total['pd2Dxz'], t['pd2Dxz'] = Dpointder3D(lehi[1], 0, 15, 16, 1, '2Dxz')
D['pd2Dyz'], SD['pd2Dyz'], total['pd2Dyz'], t['pd2Dyz'] = Dpointder3D(lehi[1], 0, 15, 16, 1, '2Dyz')

print(D['pd3D'], SD['pd3D'])
print(D['pd2Dxy'], SD['pd2Dxy'])
print(D['pd2Dxz'], SD['pd2Dxz'])
print(D['pd2Dyz'], SD['pd2Dyz'])

print(total['pd2Dxy'])

0.0411334914691 0.0446686718823
0.0441675948823 0.049881720271
0.041800515318 0.0562291772186
0.0374323642071 0.0513460317509
504


In [4]:
D['linalg3D'], SD['linalg3D'], total['linalg3D'] = Dlinalg3D(lehi[1], 0, 15, 8, '3D')
D['linalg2Dxy'], SD['linalg2Dxy'], total['linalg2Dxy'] = Dlinalg3D(lehi[1], 0, 15, 8, '2Dxy')
D['linalg2Dxz'], SD['linalg2Dxz'], total['linalg2Dxz'] = Dlinalg3D(lehi[1], 0, 15, 8, '2Dxz')
D['linalg2Dyz'], SD['linalg2Dyz'], total['linalg2Dyz'] = Dlinalg3D(lehi[1], 0, 15, 8, '2Dyz')

print(D['linalg3D'], SD['linalg3D'])
print(D['linalg2Dxy'], SD['linalg2Dxy'])
print(D['linalg2Dxz'], SD['linalg2Dxz'])
print(D['linalg2Dyz'], SD['linalg2Dyz'])

0.0698362482589 0.000994059621029
0.1014794942 0.00123767777599
0.0587900264669 0.00116860915124
0.0492392241095 0.00149733700171


In [5]:
D['linalgw3D'], SD['linalgw3D'], total['linalgw3D'] = Dlinalgw3D(nephi, 0, 15, 8, '3D')
D['linalgw2Dxy'], SD['linalgw2Dxy'], total['linalgw2Dxy'] = Dlinalgw3D(lehi[1], 0, 15, 8, '2Dxy')
D['linalgw2Dxz'], SD['linalgw2Dxz'], total['linalgw2Dxz'] = Dlinalgw3D(lehi[1], 0, 15, 8, '2Dxz')
D['linalgw2Dyz'], SD['linalgw2Dyz'], total['linalgw2Dyz'] = Dlinalgw3D(lehi[1], 0, 15, 8, '2Dyz')

print(D['linalgw3D'], SD['linalgw3D'])
print(D['linalgw2Dxy'], SD['linalgw2Dxy'])
print(D['linalgw2Dxz'], SD['linalgw2Dxz'])
print(D['linalgw2Dyz'], SD['linalgw2Dyz'])

0.0718180760453 0.00121663943009
0.101639835466 0.00112058322581
0.0602602132104 0.00136471480728
0.0527802616778 0.0015004069399


In [6]:
def plot_Mean2DMSDsorDeffLOG(traj, n1, n2, n3, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    # Creates an array 'particles' that contains the particle number at each frame.
    particles = traj[:, n1]
    total = int(max(particles))
    total1 = total + 1
    rawtime = traj[:, n2]
    bow = traj.shape[0]
    raw3DMSDs = np.zeros((bow, 5))
    raw3DMSDs[:, 0:4] = traj[:, n3: n3 + 4]
    MSD = dict()
    time = dict()

    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        MSD[num] = (raw3DMSDs[min1+1:max1, :])
        time[num] = (rawtime[min1+1:max1])

    MMSD = MSD[1]
    for num in range(2, total1):
        MMSD = MMSD + MSD[num]
    MMSD = MMSD/total1
    SD = np.zeros(np.shape(MMSD))
    
    #Now to calculate the standard dev at each point:
    for num in range (1, total1):
        SDunit = (MSD[num] - MMSD)**2
        SD = SD + SDunit
    SD = np.sqrt(SD/total1)
    SE = SD/np.sqrt(total1)
    
    #Linear algebra to find Deff:
    t = time[1][:]
    w = dict()
    line = dict()
    A = np.ones((np.shape(t)[0], 2))
    A[:, 0] = t
    w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
    w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
    w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
    w[3] = np.linalg.lstsq(A, MMSD[:, 3])[0]
    line[0] = w[0][0]*t + w[0][1]
    line[1] = w[1][0]*t + w[1][1]
    line[2] = w[2][0]*t + w[2][1]
    line[3] = w[3][0]*t + w[3][1]
    
    #Linear algebra for fit on log plot:
    wl = dict()
    linel = dict()
    lt = np.log(t)
    lA = np.ones((np.shape(t)[0], 2))
    lA[:, 0] = lt
    lM = np.log(MMSD)
    wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
    wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
    wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
    wl[3] = np.linalg.lstsq(lA, lM[:, 3])[0]
    linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
    linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
    linel[2] = np.exp(wl[2][0]*lt + wl[2][1])
    linel[3] = np.exp(wl[3][0]*lt + wl[3][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, MMSD[:, 0], linestyle='', linewidth=10, label='2D', marker='o', ms=10, color='blue')
    ax.plot(t, MMSD[:, 1], linestyle='', linewidth=10, label='2D xy', marker='o', ms=10, color='red')
    ax.plot(t, MMSD[:, 2], linestyle='', linewidth=10, label='2D xz', marker='o', ms=10, color='green')
    ax.plot(t, MMSD[:, 3], linestyle='', linewidth=10, label='2D yz', marker='o', ms=10, color='purple')
    
    ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
    ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
    ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    ax.errorbar(t, MMSD[:, 3], yerr=SE[:, 3], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='purple')
    
    #ax.plot(t, line[0], linewidth=3, color='blue')
    #ax.plot(t, line[1], linewidth=3, color='red')
    #ax.plot(t, line[2], linewidth=3, color='green')
    
    ax.plot(t, linel[0], linewidth=3, color='blue')
    ax.plot(t, linel[1], linewidth=3, color='red')
    ax.plot(t, linel[2], linewidth=3, color='green')
    ax.plot(t, linel[3], linewidth=3, color='purple')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.72), prop={'size': 40})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    plt.yscale('log')
    plt.xscale('log')
    plt.gca().set_xlim([1, limit1])
    plt.gca().set_ylim([0.01, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return MMSD, total1

In [7]:
one1, total1 = plot_Mean2DMSDsorDeffLOG(lehi[1], 0, 15, 8, 1, 1, 'MSD (um^2)', logplot, 100.1, 100.1, 1, 1)
plt.show()

In [8]:
def plot_Mean2DMSDsorDeff(traj, n1, n2, n3, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    # Creates an array 'particles' that contains the particle number at each frame.
    particles = traj[:, n1]
    total = int(max(particles))
    total1 = total + 1
    rawtime = traj[:, n2]
    bow = traj.shape[0]
    raw3DMSDs = np.zeros((bow, 5))
    raw3DMSDs[:, 0:4] = traj[:, n3: n3 + 4]
    MSD = dict()
    time = dict()

    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        MSD[num] = (raw3DMSDs[min1+1:max1, :])
        time[num] = (rawtime[min1+1:max1])

    MMSD = MSD[1]
    for num in range(2, total1):
        MMSD = MMSD + MSD[num]
    MMSD = MMSD/total1
    SD = np.zeros(np.shape(MMSD))
    
    #Now to calculate the standard dev at each point:
    for num in range (1, total1):
        SDunit = (MSD[num] - MMSD)**2
        SD = SD + SDunit
    SD = np.sqrt(SD/total1)
    SE = SD/np.sqrt(total1)
    
    #Linear algebra to find Deff:
    t = time[1][:]
    w = dict()
    line = dict()
    A = np.ones((np.shape(t)[0], 2))
    A[:, 0] = t
    w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
    w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
    w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
    w[3] = np.linalg.lstsq(A, MMSD[:, 3])[0]
    line[0] = w[0][0]*t + w[0][1]
    line[1] = w[1][0]*t + w[1][1]
    line[2] = w[2][0]*t + w[2][1]
    line[3] = w[3][0]*t + w[3][1]
    
    #Linear algebra for fit on log plot:
    wl = dict()
    linel = dict()
    lt = np.log(t)
    lA = np.ones((np.shape(t)[0], 2))
    lA[:, 0] = lt
    lM = np.log(MMSD)
    wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
    wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
    wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
    wl[3] = np.linalg.lstsq(lA, lM[:, 3])[0]
    linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
    linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
    linel[2] = np.exp(wl[2][0]*lt + wl[2][1])
    linel[3] = np.exp(wl[3][0]*lt + wl[3][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, MMSD[:, 0], linestyle='', linewidth=10, label='3D', marker='o', ms=10, color='blue')
    ax.plot(t, MMSD[:, 1], linestyle='', linewidth=10, label='2D xy', marker='o', ms=10, color='red')
    ax.plot(t, MMSD[:, 2], linestyle='', linewidth=10, label='2D xz', marker='o', ms=10, color='green')
    ax.plot(t, MMSD[:, 3], linestyle='', linewidth=10, label='2D yz', marker='o', ms=10, color='purple')
    
    ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
    ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
    ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    ax.errorbar(t, MMSD[:, 3], yerr=SE[:, 3], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='purple')
    
    ax.plot(t, line[0], linewidth=3, color='blue')
    ax.plot(t, line[1], linewidth=3, color='red')
    ax.plot(t, line[2], linewidth=3, color='green')
    ax.plot(t, line[3], linewidth=3, color='purple')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.72), prop={'size': 40})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return MMSD, total1

In [9]:
one1, total1= plot_Mean2DMSDsorDeff(lehi[1], 0, 15, 8, 0, 0, 'MSD (um^2)', Mplot, 30.1, 20.01, 5, 4)
plt.show()

In [10]:
def plot_Mean1DMSDsorDeffold(traj, n1, n2, n3, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    # Creates an array 'particles' that contains the particle number at each frame.
    particles = traj[:, n1]
    total = int(max(particles))
    total1 = total + 1
    rawtime = traj[:, n2]
    bow = traj.shape[0]
    raw3DMSDs = np.zeros((bow, 5))
    raw3DMSDs[:, 0:4] = traj[:, n3: n3 + 4]
    MSD = dict()
    time = dict()

    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        MSD[num] = (raw3DMSDs[min1+1:max1, :])
        time[num] = (rawtime[min1+1:max1])

    MMSD = MSD[1]
    for num in range(2, total1):
        MMSD = MMSD + MSD[num]
    MMSD = MMSD/total1

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(time[1][:], MMSD[:, 0], linewidth=10, label='3D')
    ax.plot(time[1][:], MMSD[:, 1], linewidth=10, label='2D xy')
    ax.plot(time[1][:], MMSD[:, 2], linewidth=10, label='2D xz')
    ax.plot(time[1][:], MMSD[:, 3], linewidth=10, label='2D yz')

    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.65, 0.05), prop={'size': 70})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return MMSD

In [15]:
one1 = plot_Mean1DMSDsorDeffold(lehi[1], 0, 15, 16, 0, 2, 'Deff (um^2/s)', Dplot, 30.1, 0.121, 5, 0.04)
plt.show()

In [12]:
def plot_histogram3D(traj, n1, n2, n3, datatype, filename, tn):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    # Creates an array 'particles' that contains the particle number at each frame.
    particles = traj[:, n1]
    total = int(max(particles))
    total1 = total + 1
    rawtime = traj[:, n2]
    bow = traj.shape[0]
    raw3DMSDs = np.zeros((bow, 5))
    raw3DMSDs[:, 0:4] = traj[:, n3: n3 + 4]
    MSD = dict()
    time = dict()

    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        MSD[num] = (raw3DMSDs[min1+1:max1, :])
        time[num] = (rawtime[min1+1:max1])
    
    t = time[1][:]
    def find_nearest(array, value):
        idx = (np.abs(array-value)).argmin()
        return array[idx], idx

    td, idx = find_nearest(t, tn)
    
    hist = np.zeros(total1)
    for num in range(1, total1):
        hist[num] = MSD[num][idx, 0]
        
    plot, bins = np.histogram(hist, bins=20)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:])/2
    plt.bar(center, plot, align='center', width=width)

    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return MSD, total1

In [13]:
hist, total= plot_histogram3D(lehi[1], 0, 15, 16, 'Deff (um^2/s)', Hplot, 1)
plt.show()